<a href="https://colab.research.google.com/github/ngkriz/NOTAMate/blob/main/NOTAMate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import os  # For folder management
import pandas as pd
import re
import openpyxl
from openpyxl.styles import PatternFill
from datetime import datetime
from google.colab import files  # Import files module for Colab

# Generate the current UTC time in yymmddhhmm format
current_time = datetime.utcnow().strftime('%y%m%d%H%M')

def process_data(input_text):
    # Split the input into blocks by separating at blank lines
    blocks = re.split(r'\n\s*\n', input_text.strip())

    # Define the fields to ensure all are included
    required_fields = ["QQQQQ", "AAAAA", "BBBBB", "CCCCC", "DDDDD", "EEEEE", "FFFFF", "GGGGG", "YYYYY", "ZZZZZ"]

    processed_blocks = []
    for block in blocks:
        # Parse lines into a dictionary
        lines = block.strip().split("\n")
        data_dict = {}
        for line in lines:
            parts = line.split(maxsplit=1)
            if len(parts) == 2:
                key, value = parts
                data_dict[key.strip()] = value.strip()

        # Ensure all required fields are present
        for field in required_fields:
            if field not in data_dict:
                data_dict[field] = "*****"

        # Reconstruct the block with all fields in order
        processed_block = "\n".join(f"{field}     {data_dict[field]}" for field in required_fields)
        processed_blocks.append(processed_block)

    # Combine all processed blocks into final output
    return "\n\n".join(processed_blocks)

# Take user input for the NOTAM text
text1 = input("Enter the NOTAM text: ")

# Default header
header = 'Q) Code  A) Area B) Start time C) End time D) Effective Time E) Details F) Base G) Top /.Q Remarks ./'
text = header + '\n' + text1

# Input for activate time with a default value
activate_time = input(f"Enter the NOTAM begin time in UTC (default NOW: {current_time}): ")
if not activate_time:
    activate_time = current_time

# Input for FL with a default value
fl_value = input("Enter the top FL value (default: 240): ")
if not fl_value:
    fl_value = 239

highlighter = input("Specific word to be highlighted in RED in column E: ")
highlighter = highlighter.upper()

print ("")
print ("Loading...")
print ("")

# Define the replacements with spacing
replacements = {
    "Q)": "\n\nQQQQQ  ",
    "A)": "AAAAA  ",
    "B)": "BBBBB  ",
    "C)": "CCCCC  ",
    "D)": "DDDDD  ",
    "E)": "EEEEE  ",
    "F)": "FFFFF  ",
    "G)": "GGGGG  ",
    "/.Q": "YYYYY  ",
    "./": "ZZZZZ  \n\n"
}

# Replace field markers with uniform tags
for old, new in replacements.items():
    text = re.sub(re.escape(old), f"\n{new}  ", text)

# Regular expression to keep only lines starting with the specified markers
pattern = r"^(QQQQQ|AAAAA|BBBBB|CCCCC|DDDDD|EEEEE|FFFFF|GGGGG|YYYYY|ZZZZZ).*$"

# Split text into lines and filter lines that match the pattern
filtered_lines = [line for line in text.splitlines() if re.match(pattern, line.strip())]

# Add a new line after 'ZZZZZ' if it exists
output_text = ""
for line in filtered_lines:
    output_text += line + "\n"
    if line.startswith("ZZZZZ"):
        output_text += "\n"

# Process the input
output = process_data(output_text)

# Function to split the NOTAM text into structured data
def split_notam_text(output_text):
    notams = []
    for notam in output_text.strip().split("\n\n"):
        fields = {}
        for line in notam.split("\n"):
            if "     " in line:
                key, value = line.split("     ", 1)
                fields[key.strip()] = value.strip()
        if fields:
            notams.append(fields)
    return notams

# Apply the function to the raw text
notam_data = split_notam_text(output)

# Create a DataFrame from the parsed data
df = pd.DataFrame(notam_data)

# Move column A (Code) to the back
df = df[[col for col in df.columns if col != 'QQQQQ'] + ['QQQQQ']]

# Replace the header row with the first row in df
df.columns = df.iloc[0]  # Set the first row as the header
df = df[1:].reset_index(drop=True)  # Remove the first row from data and reset the index

# Split the last column using '/' as the separator
last_column = df.columns[-1]  # Get the name of the last column
split_columns = df[last_column].str.split("/", expand=True)
split_columns.columns = [f"Q){i+1}" for i in range(split_columns.shape[1])]
df = pd.concat([df.drop(columns=[last_column]), split_columns], axis=1)

# Generate a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_path = "./notam/"
file_name = f"notam_data_{timestamp}.xlsx"
full_file_path = os.path.join(folder_path, file_name)

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Save the DataFrame as an Excel file
with pd.ExcelWriter(full_file_path, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="NOTAMs")
    workbook = writer.book
    sheet = workbook["NOTAMs"]

    # Apply formatting rules to specific columns
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")








    # Set the column width for all columns to 12
    for col in sheet.columns:
        col_letter = col[0].column_letter  # Get column name (e.g., 'A', 'B', etc.)
        sheet.column_dimensions[col_letter].width = 12  # Set the width of all columns to 12

    # Apply specific formatting to column 'A' (Details column)
    sheet.column_dimensions['A'].width = 6
    sheet.column_dimensions['B'].width = 22
    # Apply specific formatting to column 'E' (Details column)
    sheet.column_dimensions['E'].width = 65  # Set the width of column 'E' to 40
    # Apply specific formatting to column 'E' (Details column)
    sheet.column_dimensions['H'].width = 32  # Set the width of column 'E' to 40

        # Apply specific formatting to column 'J' (Details column)
    sheet.column_dimensions['J'].width = 6

        # Apply specific formatting to column 'K' (Details column)
    sheet.column_dimensions['K'].width = 8

        # Apply specific formatting to column 'L' (Details column)
    sheet.column_dimensions['L'].width = 4
    sheet.column_dimensions['M'].width = 6
    sheet.column_dimensions['N'].width = 4
    sheet.column_dimensions['Q'].width = 16
    sheet.column_dimensions['R'].width = 20
    sheet.column_dimensions['S'].width = 30
    sheet.cell(row=1, column=2, value="Begin time: " + activate_time)
    sheet.cell(row=1, column=5, value="Yellow [ILS, RWY, U/S, CLSD, MIL EXER];   Green [RSC];   Amber [CRFI];  Red [defined by User]")
    sheet.cell(row=1, column=7, value="> FL" + str(fl_value))
    sheet.cell(row=1, column=18, value="Q)4 Explain")
    sheet.cell(row=1, column=19, value="Q)2 Explain")
    sheet.cell(row=1, column=16, value="Q)7 > FL" + str(fl_value))














for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    # Column 'P' (index 16)
    col_p_value = row[15].value  # Column 'P' corresponds to index 15
    if col_p_value:  # Ensure the cell is not empty
        try:
            numeric_value = float(col_p_value)  # Convert to a float
            sheet.cell(row=row[0].row, column=16, value=numeric_value)  # Update cell with numeric value
        except ValueError:
            # Handle non-numeric values (set to 0 or leave blank as per requirement)
            sheet.cell(row=row[0].row, column=16, value=0)  # Replace non-numeric with 0





# Define the yellow fill style
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green
orange_fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")  # Orange

# Highlight cells in column E if they contain "ILS"
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "ILS" in col_e_value:  # Check if "ILS" is in the cell
        row[4].fill = yellow_fill  # Apply yellow fill to the cell

# Highlight cells in column E if they contain "ILS"
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "U/S" in col_e_value:  # Check if "ILS" is in the cell
        row[4].fill = yellow_fill  # Apply yellow fill to the cell

# Highlight cells in column E if they contain "ILS"
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "CLSD" in col_e_value:  # Check if "ILS" is in the cell
        row[4].fill = yellow_fill  # Apply yellow fill to the cell

# Highlight cells in column E if they contain "ILS"
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "RWY" in col_e_value:  # Check if "ILS" is in the cell
        row[4].fill = yellow_fill  # Apply yellow fill to the cell



# Highlight cells in column E if they contain "RSC" (green) or "CRFI" (orange)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value:
        if "RSC" in col_e_value:  # Check if "RSC" is in the cell
            row[4].fill = green_fill  # Apply green fill


# Highlight cells in column E if they contain "ILS"
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "MIL EXER" in col_e_value:  # Check if "ILS" is in the cell
        row[4].fill = yellow_fill  # Apply yellow fill to the cell

# Highlight cells in column E if they contain "RSC" (green) or "CRFI" (orange)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)
    if col_e_value and "CRFI" in col_e_value:  # Check if "ILS" is in the cell
         row[4].fill = orange_fill  # Apply orange fill

from openpyxl.styles import PatternFill

# Define the red fill color
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

# Only proceed with highlighting if 'highlighter' is not empty
if highlighter:  # This ensures that the input is not empty
    # Iterate through rows, starting from row 2
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        col_e_value = row[4].value  # Column 'E' corresponds to index 4 (zero-based indexing)

        if highlighter == "EMPTY":
            if not col_e_value:  # Apply red fill to empty cells
                row[4].fill = red_fill
        elif col_e_value and highlighter in col_e_value.upper():  # Check if "highlighter" string is in the cell, case insensitive
            row[4].fill = red_fill  # Apply red fill


from openpyxl.styles import Alignment

# Enable word wrap and left alignment for column E
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=5, max_col=5):  # Column E is index 5
    for cell in row:
        cell.alignment = Alignment(wrap_text=True, horizontal='left')  # Correctly assign the Alignment object







    # Highlight 'Start Time' (column C) if greater than the activate time
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Red fill

    for row in range(2, len(df) + 2):  # Starting from row 2 to skip header row
        cell = sheet.cell(row=row, column=2)  # 'Start Time' is column 2
        start_time = str(cell.value).strip()
        if start_time and len(start_time) == 10 and start_time <= activate_time:
            cell.fill = red_fill  # Apply red fill if condition is met

    # Highlight 'FL' (column G) if greater than FL input or is "UNL"
    for row in range(2, len(df) + 2):  # Starting from row 2 to skip header row
        cell = sheet.cell(row=row, column=7)  # 'FL' is column 7
        fl_value_cell = str(cell.value).strip()

        if fl_value_cell.startswith("FL"):
            fl_value_numeric = fl_value_cell[2:].strip()  # Remove "FL" and get numeric value
            if fl_value_numeric.isdigit() and int(fl_value_numeric) > int(fl_value):
                cell.fill = red_fill  # Highlight if FL > user input
            elif fl_value_cell == "UNL":
                cell.fill = red_fill  # Highlight if FL == "UNL"

    # Set the width of column 'I' to 24
    sheet.column_dimensions['J'].width = 12

    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
        # Column 'K' (index 11): Remove first letter and check
        col_k_value = row[10].value  # Column 'K' corresponds to index 10
        if col_k_value:  # Ensure the cell is not empty
            new_value = col_k_value[1:]  # Remove the first character

            if new_value.startswith(('W', 'R')):  # Check if it starts with 'W' or 'R'
                row[10].fill = red_fill  # Apply red fill to 'K'

        # Column 'M' (index 13): Check if it starts with 'N'
        col_m_value = row[12].value  # Column 'M' corresponds to index 12
        if col_m_value and col_m_value.startswith('N'):
            row[12].fill = red_fill  # Apply red fill to 'M'




for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_p_value = row[15].value  # Column 'P' corresponds to index 15 (zero-based indexing)

    if col_p_value:  # Ensure the cell is not empty
        try:
            numeric_value = float(col_p_value)  # Convert to a numeric value
            if numeric_value > float(fl_value):  # Compare with 240
                row[15].fill = red_fill  # Apply red fill if the value is greater than 240
        except ValueError:
            continue  # Ignore non-numeric values in column 'P'


# Add "Immediate attention" to column 'R' if column M starts with 'N'
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_m_value = row[12].value  # Column 'M' corresponds to index 12 (zero-based indexing)
    if col_m_value and col_m_value.startswith('N'):  # Check if it starts with 'N'
        sheet.cell(row=row[0].row, column=18, value="Immediate attention")  # Column 'R' is column 18 (1-based indexing)




# Extract LAT/LONG from column E and store in column T
lat_long_pattern = r"(\d{2}\d{2}\d{2})([NS])\s*(\d{3}\d{2}\d{2})([EW])"  # Regex for latitude and longitude

# Add the header for LAT/LONG
sheet.cell(row=1, column=20, value="LAT/LONG")  # Column 20 corresponds to column 'T'

# Iterate through the rows and extract LAT/LONG from column E (index 4)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from row 2 to skip the header row
    col_e_value = row[4].value  # Column 'E' corresponds to index 4
    if col_e_value:  # Ensure the cell is not empty
        # Search for LAT/LONG pattern and find all occurrences
        matches = re.findall(lat_long_pattern, col_e_value)  # Find all matches
        if matches:
            # For each match, format as "LAT/LONG"
            lat_long_values = [f"{match[0]}{match[1]}{match[2]}{match[3]}" for match in matches]
            # Join all LAT/LONG pairs with a comma or any other delimiter of your choice
            lat_long_string = "  ".join(lat_long_values)
            # Assign the concatenated LAT/LONG values to column T
            sheet.cell(row=row[0].row, column=20, value=lat_long_string)

# Adjust column width for LAT/LONG
sheet.column_dimensions['T'].width = 30  # Set the width of column T to 30





from openpyxl.styles import Alignment

# Iterate through rows in column E, starting from row 2 (assuming row 1 is the header)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=5, max_col=5):  # Column E corresponds to index 5
    for cell in row:
        cell.alignment = Alignment(wrap_text=True)  # Enable word wrap for the cell

# Adjust the column width to ensure text wraps properly (optional)
sheet.column_dimensions['E'].width = 80
# Adjust this width based on your needs





for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):  # Start from the second row
    col_k_value = row[10].value  # Column 'K' corresponds to index 10 (zero-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "A":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="ATM Airspace")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "C":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Communications and Surveillance")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "F":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Facilities and Services")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "G":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="GNSS")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "I":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Instruments")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "L":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Lighting Facilities")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "M":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Movement and Landing Area")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "N":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Enroute Navigation Facilities")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "P":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="ATM Traffic Procedures")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "R":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Restrictions")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "W":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Warnings")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "X":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Specified")  # Column 'S' is column 19 (1-based indexing)
    if col_k_value and len(col_k_value) > 1 and col_k_value[1] == "O":  # Check if the second letter is "L"
        sheet.cell(row=row[0].row, column=19, value="Obstacle")  # Column 'S' is column 19 (1-based indexing)

# Apply red fill to column 'S' if the value is "Warnings" or "Restrictions"
    for row in range(2, len(df) + 2):  # Starting from row 2 to skip header row
        cell = sheet.cell(row=row, column=19)  # Column 'S' is the 19th column
        if cell.value in ["Warnings", "Restrictions"]:
            cell.fill = red_fill  # Apply red fill if condition is met


from openpyxl.styles import Alignment

# Enable word wrap and left alignment for column E
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=5, max_col=5):  # Column E is index 5
    for cell in row:
        cell.alignment = Alignment(wrap_text=True, horizontal='left')  # Correctly assign the Alignment object

# Save the workbook
writer.book.save(full_file_path)


# Trigger the download
files.download(full_file_path)

# Print the DataFrame to confirm
##print(f"File saved as: {full_file_path}")
##print(df)


print ("")
print ("")
print ("***************************** Double check the number of 'Q)' and './' to count any missing entry *****************************")
print ("")
print ("")
print ("The Excel file is downloading ......")

import pandas as pd
DataframeA =pd.DataFrame (df)


import pandas as pd
DataframeA =pd.DataFrame (df)

###DataframeA


